In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os

# Add the directory above the current notebook to the system path
sys.path.append(os.path.abspath(os.path.join('..')))

In [ ]:
from process_czi import get_channel_info, get_czi_info, maxproject_for_registration
from io_images import get_images_infoframe
import os

In [ ]:
# Brain Slices
# windows = False
# folderpath = 'data'
# folderpath = "/run/user/1001/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/archive/siegegrp/AlVe/MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS"
windows = "nt" in os.name
folderpath = r"\\fs.ista.ac.at\drives\aventuri\archive\siegegrp\AlVe\MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS\raw_images"


In [ ]:
# List all files and directories in the folder
contents = os.listdir(folderpath)

print("Contents of the folder:")
for item in contents:
    print(item)

In [ ]:
infoframe = get_images_infoframe(folderpath,conditions=['Age', 'Sex', 'Animal'])

In [ ]:
infoframe

In [ ]:
example_1_filepath = infoframe['file_path'].iloc[5]
example_2_filepath = infoframe['file_path'].iloc[7]

In [ ]:
import aicspylibczi
from process_czi import _get_channel_idx,get_czi_pixel_size
import numpy as np
import tifffile

In [ ]:
save_folderpath = folderpath = r"\\fs.ista.ac.at\drives\aventuri\archive\siegegrp\AlVe\MORPHOMICS2.0_MICROGLIA_BRAIN_ATLAS\dapi_examples"
channel_name = 'DAPI'


In [ ]:
ii = 0
for czi_file in [example_1_filepath, example_2_filepath]:
    ii += 1
    if isinstance(czi_file, str):
            czi_file = aicspylibczi.CziFile(czi_file)
        # Take the correct chanel that contains the nucleus
    metadata = czi_file.meta
    correct_channel_idx = _get_channel_idx(metadata, channel_name)
    pixel_size = get_czi_pixel_size(czi_file)
    # Because z max proj is downsample
    pixel_size['x'] *= 2
    pixel_size['y'] *= 2
    bbox = czi_file.get_all_scene_bounding_boxes()
    sequences = czi_file.get_dims_shape()
    nb_seq = len(sequences)
    print("Start zmax projections")
    print(f"There are {nb_seq} tissue slices in this file")
    print(f"So, there are {nb_seq} images that will be saved as .tif files")
    os.makedirs(save_folderpath, exist_ok = True)
    s = 1

    from utils import downsample_by_2
    x_min, y_min, w, h = bbox[s].x, bbox[s].y, bbox[s].w, bbox[s].h  # Bounding box details
    for z in [10, 20]:
        mosaic = czi_file.read_mosaic(region = (x_min, y_min, w, h), 
                                        scale_factor = 1.0, 
                                        C = correct_channel_idx, 
                                        Z = z,
                                        dtype = np.uint8,
                                        # background_color = (0.0,0.0,0.0)
        )
        z_image = mosaic[0][0] # The function returns an array (1,1,y,x)
        z_image = downsample_by_2(z_image)
        z_image = np.transpose(z_image, (1, 0))
        save_filepath = os.path.join(save_folderpath, f"dapi_example_{ii}_{z}.tif")
        # Save the chunk as a TIFF file
        tifffile.imsave(save_filepath, z_image, metadata = pixel_size)
        del z_image